## How dictionaries (and sets) work?
Note that this is oversimplified.

The structure that is used is called a Hash Table. Imagine you have normal list of some size.
When you want to put some value under some key you have to calculate under which index you want to put it.
To do this we calculate hash for our key. You can invoke build-in **hash()** function to see how it works:

In [1]:
print(hash(1))
print(hash(2))
print(hash(3))
print(hash('1'))
print(hash('asdgf'))
print(hash((1,2)))
print(hash(hash))
# print(hash([1,2]))  # <- not everything is hashable - not everything can be a key in dictionary

1
2
3
-128948075
86939143
1299869600
86429053


Hash functions have many interesting properties. What we should note are two things:
1. Hash function should be calculated really fast.
2. It is hard to find two values a,b such that hash(a) == hash(b) 

Now you can use this value as an index. As you probably noticed those values are not really all that small. Does it mean that the list that is prepared has i.e. 652071602 available spots? Of course not. Some strategy is used to reduce the hash to actually usefull index. It can be for example something like this:

In [2]:
size_of_dict = 8
tmp = hash('asdgf')
index = tmp % size_of_dict
print(index)

7


(by the way, Python doesn't actually use %. It does **index = tmp & mask**, where mask is something somehow related to size, but that is not important)

The dictionary, even if created empty, will prepare list of some initial size to store your values. 

In [3]:
import sys
d = {}
print(sys.getsizeof(d))
for i in range(100):
    d[i] = i
    if i % 10 == 0:
        print("After adding {} the size is {}".format(i, sys.getsizeof(d)))

136
After adding 0 the size is 136
After adding 10 the size is 352
After adding 20 the size is 352
After adding 30 the size is 636
After adding 40 the size is 636
After adding 50 the size is 1216
After adding 60 the size is 1216
After adding 70 the size is 1216
After adding 80 the size is 1216
After adding 90 the size is 2620


The number we see as the size is not how many cells there are in the list, but the size of all the data. The list does not store the actuall value we want to remember but the memory address of our thing we want to store. Therefore it is a number of fixed size. I found somewhere on the Internet that there are 24 bytes reserved for each value. Then there should be

In [4]:
print(sys.getsizeof({})/24, "cells at the beginning")

5.666666666666667 cells at the beginning


When I run it I got a number that was not divisible by 24. It may vary depending on many things, such as computer architecture (32,64-bit) and so on. Anyway, it means, we can do something like this to get an usefull index.

In [5]:
tmp = hash('asdgf')
index = tmp % int(sys.getsizeof({})/24)
print(index)
tmp = hash('asfht')
index = tmp % int(sys.getsizeof({})/24)
print(index)

3
0


As you should notice, the size of the dictionary goes up at some points. Obviously, when we are putting more and more values in, we have to prepare more space for upcomming values (or rather, memory addressess).

There are still several points that complicates a lot of things. First of all, what should happen when **hash() % size** of two values fall into the same place? How the resize of the hash table is implemented, and so on. In the end we say we have O(1) complexity to inser element but it looks like a lot of stuff to do!

In case of collision (two keys hash to the same value) there is of course some strategy that describes what to do and next index is calculated in some way.

When we have to resize the list, we have to copy what we had untill now and prepare some more space.

To oversimplify, lets say that there is a lot to do only from time to time. The assumption is that even if we have to do lets say O(N) operations to insert just one element (where N is the current size of dict) we have to make those operations count! If we have to make N operations **once** per N inserts then we can say that inserting N elements requires O(N) operations, therefore one insert takes O(N)/N = O(1) time. The word associated with such analisys is "amortized" run time.  

To be honest, this is such a better answer: https://www.youtube.com/watch?v=DM-UnV9TEzY